![](http://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/UPM/EscPolitecnica/EscUpmPolit_p.gif "UPM")

# Trabajo final SITC
## Análisis de sentimientos en Twitter

Departamento de Ingeniería de Sistemas Telemáticos. Universidad Politécnica de Madrid.

Realizado por:
- Juan Bermudo Mera
- Margarita Bolívar Jiménez
- Lourdes Fernández Nieto
- Ramón Pérez Hernández

© 2017

# Algoritmo BernoulliNB aplicado sobre el fichero de tweets

## Tabla de contenidos

* [Importación de datos necesarios para aplicar el algoritmo](#1.-Importación-de-datos-necesarios-para-aplicar-el-algoritmo)
	* [Importación de librerías](#Importación-de-librerías)
    * [Importación de corpus y tweets](#Importación-de-corpus-y-tweets)
    * [Tokenización y stemming](#Tokenización-y-stemming)
* [Entrenamiento del modelo](#2.-Entrenamiento-del-modelo)
* [Rendimiento del modelo](#3.-Rendimiento-del-modelo)
* [Predicción de la polaridad](#4.-Predicción-de-la-polaridad)

## 1. Importación de datos necesarios para aplicar el algoritmo

* ### Importación de librerías

In [1]:
# Importamos librerías. Las que no están instaladas, instalar con pip install <nombre_paquete>
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import SnowballStemmer

/Users/Lourdes/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/Lourdes/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


* ### Importación de corpus y tweets

In [2]:
# Se importan el corpus y los tweets
tweets_df = pd.read_excel('ficheros/Preprocesados/tweets_corpus_header.xlsx', header=0, encoding='iso8859_15')
tweets = pd.read_excel('ficheros/TweetsConTopic/tweets_spainGeo_topic.xlsx', header=0, encoding='iso8859_15')

* ### Tokenización y stemming

In [3]:
# Se descargan las palabras de parada en español
nltk.download("stopwords")
spanish_stopwords = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Lourdes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Obtenemos los signos de puntuación que se utilizan en español
non_words = list(punctuation)
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))

In [5]:
# Se definen las funciones para realizar la tokenización y el stemming
stemmer = SnowballStemmer('spanish')
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    # Eliminamos lo que no sean palabras
    text = ''.join([c for c in text if c not in non_words])
    # Tokenización
    tokens = tknzr.tokenize(text)

    # Stemming
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems

## 2. Entrenamiento del modelo

In [6]:
# Buscamos los parámetros que podemos utilizar para entrenar el modelo
BernoulliNB().get_params()

{'alpha': 1.0, 'binarize': 0.0, 'class_prior': None, 'fit_prior': True}

In [9]:
# Creamos el pipeline para poder encadenar todos los elementos necesarios para realizar la estimación.
# Realizamos la búsqueda mediante GridSearhCV, que es una librería de sklearn que permite realizar una
# búsqueda de los mejores parámetros del modelo, utulizando los parámetros definidos en parameters y como
# métrica, roc_auc (área bajo la curva ROC)
pipeline = Pipeline([
    ('vect',  CountVectorizer(
                analyzer = 'word',
                tokenizer = tokenize,
                lowercase = True,
                stop_words = spanish_stopwords)),
    ('cls', BernoulliNB())])
parameters = {
    'cls__alpha': (0.001, 0.01, 0.1, 1),
    'cls__fit_prior': ('True', 'False')
}
gs = GridSearchCV(pipeline, parameters, n_jobs=-1, scoring='roc_auc')
gs.fit(tweets_df.content, tweets_df.polarity_bin)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['de', 'la'... vocabulary=None)), ('cls', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'cls__alpha': (0.001, 0.01, 0.1, 1), 'cls__fit_prior': ('True', 'False')},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [10]:
# Mostramos los mejores parámetros del SVC obtenidos de la búsqueda con GridSearchCV 
gs.best_params_

{'cls__alpha': 1, 'cls__fit_prior': 'True'}

El mejor parámetro que encuentra GridSearchCV para BernoulliNB es de alplha = 1 y fit_prior = True

In [11]:
# Guardamos el resultado del GridSearchCV en un fichero de manera persistente
from sklearn.externals import joblib
joblib.dump(gs, 'ficheros/ResultadosGridSearch/grid_searchBernoulliNB.pkl')

['ficheros/ResultadosGridSearch/grid_searchBernoulliNB.pkl']


## 3. Rendimiento del modelo

In [13]:
# Mediante validación cruzada obtenemos el rendimiento del modelo
model = BernoulliNB(alpha=1, fit_prior = 'True')

vectorizer = CountVectorizer(
    analyzer = 'word',
    tokenizer = tokenize,
    lowercase = True,
    stop_words = spanish_stopwords,
    min_df = 0,
    max_df = 4701,
    max_features=1000
)

tweets_df_data_features = vectorizer.fit_transform(tweets_df.content)
tweets_df_data_features_nd = tweets_df_data_features.toarray()

scores = cross_val_score(
    model,
    tweets_df_data_features_nd[0:len(tweets_df)],
    y=tweets_df.polarity_bin,
    scoring='roc_auc',
    cv=None
    )

scores.mean()

0.85423704792147692

El valor que se obtiene del rendimiento del modelo para la métrica de Área bajo la Curva ROC es de 0.85423704792147692

## 4. Predicción de la polaridad

In [6]:
# Una vez que tenemos el modelo que mejor métrica nos aporta (tras realizar muchas pruebas con distintas métricas 
# y parámetros pasados al modelo), volvemos a crear un pipeline pero en este caso, pasándole los mejores parámetros
# obtenidos del SVC para predecir qué polaridad tienen los tweets que están aún sin etiquetar
pipeline = Pipeline([
    ('vect', CountVectorizer(
            analyzer = 'word',
            tokenizer = tokenize,
            lowercase = True,
            stop_words = spanish_stopwords,
            min_df = 0,
            max_df = 26363,
            max_features=1000
            )),
    ('cls', BernoulliNB(alpha=1, fit_prior = 'True'))
])

pipeline.fit(tweets_df.content, tweets_df.polarity_bin)
tweets['polarity'] = pipeline.predict(tweets.content)

In [7]:
# Mostramos algunos de los tweets que han sido etiquetados con la polaridad
tweets[['content', 'polarity','Topic']].sample(20)

,content,polarity,Topic
24625,#Madrid #SINFILTROS #Dibujo #Draw #MadridDesco...,1,otros
21868,@davidlabola las autonómías tienen que recorta...,0,otros
18455,"How big, How ginger, How @hanabolabola Pues he...",1,otros
5998,Hoy desayunamos con la taza de la peli #losAmo...,1,otros
20858,"Nueva Colección en Casual&amp;Chic,La Condesa,...",1,otros
23186,"""Lo amigos son la familia que se escoge. @ Tor...",1,otros
23384,las #nubes tienen un amplio rango de bellos gr...,1,otros
13104,I'm at Parque Empresarial La Finca in Pozuelo ...,1,sitios-madrid
1706,Nuestro último post ️ Los 8 impresionantes ben...,1,otros
17188,Gran final sanseña! Premio al mejor lector y a...,1,otros


In [8]:
# Guardamos los tweets en un fichero csv con su polaridad
tweets[[ 'content', 'Latitude', 'Longitude', 'polarity','Topic']].to_csv('ficheros/TweetsConPolaridadYTopic/tweetsBernoulliNB_polarity_bin.csv', encoding='utf-8')

In [9]:
# Guardamos los tweets en fichero excel
tweets[['content','Latitude','Longitude','polarity','Topic']].to_excel('ficheros/TweetsConPolaridadYTopicExcel/tweetsBernoulliNB_polarity_bin.xlsx', header=True, index=False)

<hr>

## Licencia

El notebook está licenciado libremente bajo la licencia [Creative Commons Attribution Share-Alike](https://creativecommons.org/licenses/by/2.0/).

La base del código empleado procede del trabajo de Manuel Garrido llamado [Cómo hacer Análisis de Sentimiento en español](http://pybonacci.org/2015/11/24/como-hacer-analisis-de-sentimiento-en-espanol-2/).

© 2017 - Juan Bermudo Mera, Margarita Bolívar Jiménez, Lourdes Fernández Nieto, Ramón Pérez Hernández.

Universidad Politécnica de Madrid.